In [1]:


import os
import pickle
from typing import Dict, List

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model


# ==============================
# Custom metric (needed for loading models)
# ==============================
@tf.keras.utils.register_keras_serializable(
    package="Custom",
    name="symmetric_mean_absolute_percentage_error"
)
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    numerator = tf.abs(y_true - y_pred)
    denominator = (tf.abs(y_true) + tf.abs(y_pred)) / 2
    smape = tf.reduce_mean(numerator / denominator) * 100
    return smape


# ==============================
# Model & Preprocessor Loader
# ==============================
class ModelLoader:
    def __init__(self, model_paths: Dict[str, str], preprocessor_paths: Dict[str, str]):
        self.model_paths = model_paths
        self.preprocessor_paths = preprocessor_paths
        self.models: Dict[str, tf.keras.Model] = {}
        self.preprocessors: Dict[str, object] = {}

    def load_models(self):
        for name, path in self.model_paths.items():
            if not os.path.exists(path):
                raise FileNotFoundError(f"Model file not found for {name}: {path}")
            self.models[name] = load_model(path)
            print(f"[INFO] Loaded model: {name} from {path}")

    def load_preprocessors(self):
        for name, path in self.preprocessor_paths.items():
            if not os.path.exists(path):
                raise FileNotFoundError(f"Preprocessor file not found for {name}: {path}")
            with open(path, "rb") as f:
                self.preprocessors[name] = pickle.load(f)
            print(f"[INFO] Loaded preprocessor: {name} from {path}")

    def get_model(self, name: str):
        model = self.models.get(name)
        if model is None:
            raise ValueError(f"Model for {name} not loaded.")
        return model

    def get_preprocessor(self, name: str):
        preprocessor = self.preprocessors.get(name)
        if preprocessor is None:
            raise ValueError(f"Preprocessor for {name} not loaded.")
        return preprocessor


# ==============================
# Paths for models & preprocessors
# ==============================
MODEL_PATHS = {
    "google": "outputs/stock_price_rnn_google_model.keras",
    "meta": "outputs/stock_price_rnn_meta_model.keras",
    "apple": "outputs/stock_price_lstm_apple_model.keras",
    "nvidia": "outputs/stock_price_lstm_nvidia_model.keras",
}

PREPROCESSOR_PATHS = {
    "google": "outputs/google_scale.pkl",
    "meta": "outputs/meta_scale.pkl",
    "apple": "outputs/apple_scale.pkl",
    "nvidia": "outputs/nvidia_scale.pkl",
}


# ==============================
# Validation Helper
# ==============================
def validate_input_series(series: List[float], name: str, expected_length: int = 60):
    if not isinstance(series, list):
        raise TypeError(f"{name} series must be a list of floats.")
    if len(series) != expected_length:
        raise ValueError(
            f"{name} time series must have exactly {expected_length} elements "
            f"(got {len(series)})."
        )
    # Optional: check all items are numeric
    for v in series:
        if not isinstance(v, (int, float)):
            raise TypeError(f"All values in {name} series must be numeric (int/float).")


def validate_input_data(data: Dict[str, List[float]]):
    required_keys = ["google", "meta", "apple", "nvidia"]
    for key in required_keys:
        if key not in data:
            raise KeyError(f"Missing key in input data: '{key}'")
        validate_input_series(data[key], key)


# ==============================
# Prediction Function
# ==============================
def predict_stock_prices(
    model_loader: ModelLoader, data: Dict[str, List[float]]
) -> Dict[str, float]:
    """
    data format:
    {
        "google": [60 floats],
        "meta": [60 floats],
        "apple": [60 floats],
        "nvidia": [60 floats]
    }
    """
    # Validate structure and lengths
    validate_input_data(data)

    results: Dict[str, float] = {}
    companies = ["google", "meta", "apple", "nvidia"]

    for company in companies:
        input_series = np.array(data[company]).reshape(-1, 1)  # shape: (60, 1)

        preprocessor = model_loader.get_preprocessor(company)
        model = model_loader.get_model(company)

        # Scale, reshape for model input
        scaled_series = preprocessor.transform(input_series)  # (60, 1)
        scaled_series = scaled_series.reshape(1, -1, 1)       # (1, 60, 1)

        # Predict
        prediction = model.predict(scaled_series)
        # Inverse transform to original scale
        predicted_price = preprocessor.inverse_transform(prediction)[0][0]
        results[f"predicted_close_price_{company}"] = float(predicted_price)

    return results


# ==============================
# Main (example usage)
# ==============================
if __name__ == "__main__":
    # 1. Load models & preprocessors
    model_loader = ModelLoader(MODEL_PATHS, PREPROCESSOR_PATHS)
    model_loader.load_models()
    model_loader.load_preprocessors()

    # 2. Example dummy input (replace with real last 60 closing prices)
    example_data = {
        "google": [100.0] * 60,
        "meta": [200.0] * 60,
        "apple": [150.0] * 60,
        "nvidia": [300.0] * 60,
    }

    # 3. Get predictions
    predictions = predict_stock_prices(model_loader, example_data)

    print("\nPredicted next-day closing prices:")
    for k, v in predictions.items():
        print(f"{k}: {v:.4f}")


[INFO] Loaded model: google from outputs/stock_price_rnn_google_model.keras
[INFO] Loaded model: meta from outputs/stock_price_rnn_meta_model.keras
[INFO] Loaded model: apple from outputs/stock_price_lstm_apple_model.keras
[INFO] Loaded model: nvidia from outputs/stock_price_lstm_nvidia_model.keras
[INFO] Loaded preprocessor: google from outputs/google_scale.pkl
[INFO] Loaded preprocessor: meta from outputs/meta_scale.pkl
[INFO] Loaded preprocessor: apple from outputs/apple_scale.pkl
[INFO] Loaded preprocessor: nvidia from outputs/nvidia_scale.pkl
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step

Predicted next-day closing prices:
predicted_close_price_google: 99.4021
predicted_close_price_meta: 186.3513
predicted_close_price_apple: 150.5722
predicted_close_price_nvidia: 333.5320
